In [1]:
from model import EncoderCNN, WordRNN, SentenceRNN
from dataLoader import build_vocab, get_loader
import dataLoader
import torch
import torch.nn as nn
import torch.utils.data as data
from torchvision import transforms
import pickle
import datetime
import os
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
%load_ext autoreload
%autoreload 2


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
torch.cuda.get_device_properties(device).total_memory

11996954624

In [6]:
torch.cuda.empty_cache()

In [7]:
torch.cuda.memory_allocated(device=None)

0

In [15]:
class Im2pGenerator(object):
    def __init__(self, img_embed_size, input_size, hidden_size, topic_size, embed_size, batch_size, learning_rate, lambda_word, lambda_sent):
        self.encoderCnn = self.__init_encoderCNN(img_embed_size)
        self.sentRnn = self.__init_sentenceRNN(input_size, hidden_size, topic_size)
        self.vocab = self.__init_vocab()
        vocab_size = len(self.vocab)
        self.wordRnn = self.__init_wordRNN(embed_size, hidden_size, vocab_size)
        self.transform = self.__init_transform()
        self.criterion = self.__init_criterion()
        params = list(self.encoderCnn.parameters()) + list(self.sentRnn.parameters()) + list(self.wordRnn.parameters())
        self.optimizer = self.__init_optimizer(learning_rate, params)
        self.train_dataLoader = self.__init_data_loader(self.vocab, self.transform, batch_size, "data/small_train.json")
        self.val_dataLoader = self.__init_data_loader(self.vocab, self.transform, batch_size, "data/val_split.json")
        self.lambda_word = lambda_word
        self.lambda_sent = lambda_sent
        self.scheduler = self.__init_scheduler()
        self.min_loss = 10000000
        self.model_path = ''
        
    
    def train(self, num_epochs):
        for epoch in range(1, num_epochs+1):
            train_loss = self.__epoch_train()
            val_loss = 0
            #val_loss = self.__epoch_val()
            self.scheduler.step(train_loss)
            print("[{}] Epoch-{} - train loss:{} - val loss:{} - lr:{}".format(datetime.datetime.now(),
                                                                               epoch + 1,
                                                                               train_loss,
                                                                               val_loss,
                                                                               self.optimizer.param_groups[0]['lr']))
            self.save_model(train_loss, "model1")
    
    def __epoch_train(self):
        self.encoderCnn.train()
        self.sentRnn.train()
        self.wordRnn.train()
        train_loss = 0
        
        for i,(images, image_id, target, prob_true) in enumerate(self.train_dataLoader):
            images = images.to(device)
            features = self.encoderCnn(images)
            prob_true = prob_true.long().to(device)
            self.optimizer.zero_grad()
            hiddens = None
            sentLoss = 0
            wordLoss = 0
            for sent_index in range(target.shape[1]):
                probs, topic, hiddens = self.sentRnn.forward(features = features, states = hiddens)
                target_cur = target[:,sent_index,:].to(device)
                print('prob true',prob_true[:, sent_index].view(-1,1))
                sentLoss += self.criterion(probs[:,0,:], prob_true[:, sent_index]).sum()
                for word_index in range(1, target_cur.shape[1]-1):
                    outputs = self.wordRnn.forward(topic, target_cur[:, :word_index])
                    captionMask = (target_cur[:, word_index] > 1).float() 
                    time_loss = self.criterion(outputs, target_cur[:, word_index].view(-1,))
                    time_loss = time_loss * captionMask
                    wordLoss += time_loss.sum()
            loss = self.lambda_word * wordLoss + self.lambda_sent * sentLoss
            loss.backward()
            self.optimizer.step()
            train_loss += loss.item()
                
        return train_loss
            
    
    def __epoch_val(self):
        self.encoderCnn.eval()
        self.sentRnn.eval()
        self.wordRnn.eval()
        val_loss = 0
        
        for i,(images, image_id, target, prob_true) in enumerate(self.val_dataLoader):
            images = images.to(device)
            features = self.encoderCnn(images)
            prob_true = prob_true.long().to(device)
            hiddens = None
            sentLoss = 0
            wordLoss = 0
            loss = 0
            for sent_index in range(target.shape[1]):
                probs, topic, hiddens = self.sentRnn.forward(features = features, hiddens = hiddens)
                target_cur = target[:,sent_index,:].to(device)
                print('prob true',prob_true[:, sent_index].view(-1,1))
                sentLoss += self.criterion(probs[:,0,:], prob_true[:, sent_index].view(-1,1)).sum()
                for word_index in range(1, target_cur.shape[1]-1):
                    outputs = self.wordRnn.forward(topic, target_cur[:, :word_index])
                    captionMask = (target_cur[:, word_index] > 1).float() 
                    time_loss = self.criterion(outputs, target_cur[:, word_index].view(-1,))
                    time_loss = time_loss * captionMask
                    wordLoss += time_loss.sum()
            loss = self.lambda_word * wordLoss + self.lambda_sent * sentLoss
            val_loss += loss.item()
                
        return val_loss
    
    
    
    def __init_data_loader(self, vocab, transform, batch_size, data_json):
        image_dir = 'data/images'
        caption_json = 'data/captions.json'
        data_json = data_json
        data_loader = get_loader(image_dir=image_dir,
                         caption_json=caption_json,
                         data_json=data_json,
                         vocabulary=vocab,
                         transform=transform,
                         batch_size=batch_size,
                         shuffle=True)
        return data_loader
    
    def __init_scheduler(self):
        scheduler = ReduceLROnPlateau(self.optimizer, 'min', patience=10)
        return scheduler
    
    def __init_transform(self):
        
        resize = 256
        crop_size = 224
        transform = transforms.Compose([
        transforms.Resize(resize),
        transforms.RandomCrop(crop_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406),
                             (0.229, 0.224, 0.225))])
        return transform
    
    def __init_vocab(self):
        with open('data/vocab.pkl', 'rb') as f:
            vocab = pickle.load(f)
        return vocab
    
    def __init_encoderCNN(self, img_embed_size):
        return EncoderCNN(img_embed_size).to(device)
    
    def __init_sentenceRNN(self, input_size, hidden_size, topic_size):
        return SentenceRNN(input_size, hidden_size, topic_size).to(device)
        
    def __init_wordRNN(self, embed_size, hidden_size, vocab_size):
        return WordRNN(embed_size, hidden_size, vocab_size).to(device)
    
    def __log(self, train_loss, val_loss, epoch):
        pass
    
    def save_model(self, loss, file_name):
        if loss < self.min_loss:
            print("Saved Model in {}".format(file_name))
            torch.save({'encoderCNN': self.encoderCnn.state_dict(),
                        'sentenceRNN': self.sentRnn.state_dict(),
                        'wordRNN': self.wordRnn.state_dict(),
                        'best_loss': loss,
                        'optimizer': self.optimizer.state_dict()},
                       os.path.join(self.model_path, "{}.npz".format(file_name)))
            self.min_loss = loss
    
    def __init_logger(self):
        pass
    
    def __init_criterion(self):
        return nn.CrossEntropyLoss(reduction = 'none').cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss(reduction = 'none')
    
    def __init_optimizer(self, learning_rate, params):
        return torch.optim.Adam(params, lr=learning_rate)

In [16]:
hidden_size = 512
img_embed_size = 512
input_size = 512
topic_size = 512
embed_size = 512
batch_size = 32
lambda_word = 1
lambda_sent = 5


im2p = Im2pGenerator(img_embed_size, input_size, hidden_size, topic_size, embed_size, batch_size, 1e-3, lambda_word,lambda_sent)

In [17]:
#im2p.encoderCnn.load_state_dict(torch.load("model1.npz")['encoderCNN'], strict=False)
#im2p.sentRnn.load_state_dict(torch.load("model1.npz")['sentenceRNN'],strict=False)
#im2p.wordRnn.load_state_dict(torch.load("model1.npz")['wordRNN'], strict=False)

In [18]:
len(im2p.train_dataLoader.dataset)

506

In [19]:
im2p.train(4)

hiddens tensor([[[-0.0222, -0.0039, -0.0974,  ...,  0.1221, -0.0751, -0.0871]],

        [[-0.0967,  0.0691, -0.0770,  ...,  0.0337, -0.1036,  0.0262]],

        [[ 0.0456,  0.1234,  0.0678,  ...,  0.0117, -0.0599,  0.0862]],

        ...,

        [[-0.2112, -0.0363,  0.0644,  ..., -0.0311,  0.0171,  0.0537]],

        [[-0.0697,  0.0929, -0.0454,  ...,  0.1071,  0.0889, -0.0617]],

        [[-0.0409,  0.0116, -0.1269,  ...,  0.1236, -0.0032, -0.0571]]],
       device='cuda:0', grad_fn=<CudnnRnnBackward>)
states (tensor([[[-0.0222, -0.0039, -0.0974,  ...,  0.1221, -0.0751, -0.0871],
         [-0.0967,  0.0691, -0.0770,  ...,  0.0337, -0.1036,  0.0262],
         [ 0.0456,  0.1234,  0.0678,  ...,  0.0117, -0.0599,  0.0862],
         ...,
         [-0.2112, -0.0363,  0.0644,  ..., -0.0311,  0.0171,  0.0537],
         [-0.0697,  0.0929, -0.0454,  ...,  0.1071,  0.0889, -0.0617],
         [-0.0409,  0.0116, -0.1269,  ...,  0.1236, -0.0032, -0.0571]]],
       device='cuda:0', grad_fn=<Cudnn

states (tensor([[[-0.0137,  0.0358, -0.2183,  ...,  0.2494, -0.1982, -0.1574],
         [-0.1581,  0.0902, -0.1250,  ...,  0.0268, -0.2037,  0.0227],
         [ 0.0888,  0.2448,  0.1403,  ..., -0.0260, -0.0891,  0.1445],
         ...,
         [-0.3294, -0.0835,  0.1357,  ..., -0.0582,  0.0516,  0.0931],
         [-0.1344,  0.1457, -0.1150,  ...,  0.2166,  0.1769, -0.0931],
         [-0.0491,  0.0442, -0.1979,  ...,  0.2254, -0.0191, -0.1313]]],
       device='cuda:0', grad_fn=<CudnnRnnBackward>), tensor([[[-0.0227,  0.0664, -0.7111,  ...,  0.4737, -0.5425, -0.4034],
         [-0.2865,  0.2149, -0.3237,  ...,  0.0851, -0.4408,  0.0405],
         [ 0.2067,  0.6917,  0.2508,  ..., -0.0512, -0.1980,  0.2733],
         ...,
         [-0.8308, -0.1993,  0.2642,  ..., -0.0992,  0.1133,  0.2440],
         [-0.3136,  0.3453, -0.1936,  ...,  0.4310,  0.4170, -0.1917],
         [-0.1321,  0.1214, -0.4130,  ...,  0.4546, -0.0410, -0.3269]]],
       device='cuda:0', grad_fn=<CudnnRnnBackward>))
pr

hiddens tensor([[[ 0.0174, -0.0900,  0.0282,  ...,  0.0453, -0.1857,  0.0259]],

        [[ 0.1164,  0.0507,  0.1260,  ...,  0.0510,  0.0286,  0.0857]],

        [[ 0.0851, -0.0232,  0.1233,  ..., -0.0172,  0.0108,  0.0857]],

        ...,

        [[-0.2297,  0.0081, -0.0839,  ..., -0.0566, -0.0851, -0.0986]],

        [[ 0.0993,  0.0967, -0.0753,  ...,  0.0061, -0.1013,  0.0111]],

        [[-0.0737,  0.0041,  0.1003,  ..., -0.0233,  0.0086,  0.1057]]],
       device='cuda:0', grad_fn=<CudnnRnnBackward>)
states (tensor([[[ 0.0174, -0.0900,  0.0282,  ...,  0.0453, -0.1857,  0.0259],
         [ 0.1164,  0.0507,  0.1260,  ...,  0.0510,  0.0286,  0.0857],
         [ 0.0851, -0.0232,  0.1233,  ..., -0.0172,  0.0108,  0.0857],
         ...,
         [-0.2297,  0.0081, -0.0839,  ..., -0.0566, -0.0851, -0.0986],
         [ 0.0993,  0.0967, -0.0753,  ...,  0.0061, -0.1013,  0.0111],
         [-0.0737,  0.0041,  0.1003,  ..., -0.0233,  0.0086,  0.1057]]],
       device='cuda:0', grad_fn=<Cudnn

hiddens tensor([[[ 0.0760, -0.1186,  0.0356,  ...,  0.0455, -0.3464,  0.0811]],

        [[ 0.1841,  0.0615,  0.2421,  ...,  0.1101,  0.0339,  0.1492]],

        [[ 0.1479, -0.0610,  0.2134,  ..., -0.0192,  0.0293,  0.1783]],

        ...,

        [[-0.3756,  0.0324, -0.2052,  ..., -0.0489, -0.1666, -0.1461]],

        [[ 0.2210,  0.2671, -0.1328,  ...,  0.0246, -0.2359,  0.0453]],

        [[-0.1417, -0.0169,  0.1963,  ..., -0.0275,  0.0182,  0.1962]]],
       device='cuda:0', grad_fn=<CudnnRnnBackward>)
states (tensor([[[ 0.0760, -0.1186,  0.0356,  ...,  0.0455, -0.3464,  0.0811],
         [ 0.1841,  0.0615,  0.2421,  ...,  0.1101,  0.0339,  0.1492],
         [ 0.1479, -0.0610,  0.2134,  ..., -0.0192,  0.0293,  0.1783],
         ...,
         [-0.3756,  0.0324, -0.2052,  ..., -0.0489, -0.1666, -0.1461],
         [ 0.2210,  0.2671, -0.1328,  ...,  0.0246, -0.2359,  0.0453],
         [-0.1417, -0.0169,  0.1963,  ..., -0.0275,  0.0182,  0.1962]]],
       device='cuda:0', grad_fn=<Cudnn

hiddens tensor([[[-0.1072, -0.0095, -0.0754,  ..., -0.0641,  0.0041,  0.0779]],

        [[-0.0870,  0.0533,  0.1843,  ...,  0.0356,  0.0211, -0.0816]],

        [[-0.1481,  0.1023,  0.0935,  ...,  0.1034, -0.0009, -0.0353]],

        ...,

        [[-0.1195,  0.0019,  0.1128,  ...,  0.1271, -0.0183, -0.1092]],

        [[ 0.0683,  0.0372, -0.0407,  ..., -0.0140, -0.0885,  0.0808]],

        [[-0.1217, -0.0056, -0.0166,  ...,  0.0813, -0.0485, -0.0379]]],
       device='cuda:0', grad_fn=<CudnnRnnBackward>)
states (tensor([[[-0.1072, -0.0095, -0.0754,  ..., -0.0641,  0.0041,  0.0779],
         [-0.0870,  0.0533,  0.1843,  ...,  0.0356,  0.0211, -0.0816],
         [-0.1481,  0.1023,  0.0935,  ...,  0.1034, -0.0009, -0.0353],
         ...,
         [-0.1195,  0.0019,  0.1128,  ...,  0.1271, -0.0183, -0.1092],
         [ 0.0683,  0.0372, -0.0407,  ..., -0.0140, -0.0885,  0.0808],
         [-0.1217, -0.0056, -0.0166,  ...,  0.0813, -0.0485, -0.0379]]],
       device='cuda:0', grad_fn=<Cudnn

...

hiddens tensor([[[-7.9878e-01,  7.6531e-02,  3.1311e-02,  ..., -6.5765e-03,
          -9.2413e-01,  9.1563e-02]],

        [[-9.9593e-01,  4.8173e-01,  7.2485e-01,  ..., -4.3448e-04,
          -7.8888e-02,  2.6023e-01]],

        [[-7.7492e-01,  4.2350e-01,  1.3172e-01,  ...,  7.5452e-01,
          -9.2160e-01, -6.2447e-04]],

        ...,

        [[-3.5570e-01,  9.7696e-01,  8.7108e-01,  ...,  8.4407e-01,
          -4.7082e-01,  7.0445e-01]],

        [[-9.9640e-01,  9.8592e-01,  9.9698e-01,  ...,  2.0428e-02,
           2.0498e-04,  7.0434e-01]],

        [[ 4.4956e-01,  6.2690e-02,  2.9338e-03,  ...,  1.6916e-01,
          -9.6325e-01, -3.9889e-03]]], device='cuda:0',
       grad_fn=<CudnnRnnBackward>)
states (tensor([[[-7.9878e-01,  7.6531e-02,  3.1311e-02,  ..., -6.5765e-03,
          -9.2413e-01,  9.1563e-02],
         [-9.9593e-01,  4.8173e-01,  7.2485e-01,  ..., -4.3448e-04,
          -7.8888e-02,  2.6023e-01],
         [-7.7492e-01,  4.2350e-01,  1.3172e-01,  ...,  7.5452e-01

hiddens tensor([[[-2.2766e-01,  8.8902e-01,  9.6123e-01,  ...,  7.5157e-01,
          -1.7532e-02,  8.0987e-01]],

        [[-7.0582e-01,  3.4834e-02, -3.5774e-02,  ..., -5.5333e-02,
          -6.7675e-01, -1.6353e-02]],

        [[-9.2652e-01,  5.9119e-01,  7.8108e-01,  ..., -1.6590e-02,
          -2.3095e-03,  3.4118e-01]],

        ...,

        [[-9.7149e-01,  9.3158e-01,  9.7365e-01,  ...,  2.2815e-02,
           2.8206e-04,  2.3337e-01]],

        [[-7.1067e-01, -3.8013e-03, -9.4482e-04,  ...,  2.2789e-02,
          -9.5886e-01, -2.8068e-04]],

        [[-1.2888e-01,  6.8663e-01,  5.7066e-01,  ...,  9.2364e-01,
          -9.1196e-01, -1.7951e-02]]], device='cuda:0',
       grad_fn=<CudnnRnnBackward>)
states (tensor([[[-2.2766e-01,  8.8902e-01,  9.6123e-01,  ...,  7.5157e-01,
          -1.7532e-02,  8.0987e-01],
         [-7.0582e-01,  3.4834e-02, -3.5774e-02,  ..., -5.5333e-02,
          -6.7675e-01, -1.6353e-02],
         [-9.2652e-01,  5.9119e-01,  7.8108e-01,  ..., -1.6590e-02

hiddens tensor([[[-6.9046e-01,  9.6660e-01,  9.8648e-01,  ...,  8.2474e-01,
          -3.4764e-02,  9.1543e-01]],

        [[-9.7898e-01,  3.6038e-01,  1.1290e-01,  ..., -1.9539e-03,
          -8.0018e-01, -6.8025e-04]],

        [[-9.9544e-01,  8.0836e-01,  9.0283e-01,  ..., -2.8582e-03,
          -4.8905e-02,  2.4179e-01]],

        ...,

        [[-9.9728e-01,  9.6133e-01,  9.9255e-01,  ...,  2.0997e-02,
          -1.5141e-03,  2.2059e-01]],

        [[-9.6775e-01,  1.4239e-03, -1.0593e-03,  ...,  9.1559e-03,
          -9.7778e-01, -1.7425e-04]],

        [[-5.8808e-01,  8.0160e-01,  6.9580e-01,  ...,  9.1440e-01,
          -9.4901e-01, -7.4842e-03]]], device='cuda:0',
       grad_fn=<CudnnRnnBackward>)
states (tensor([[[-6.9046e-01,  9.6660e-01,  9.8648e-01,  ...,  8.2474e-01,
          -3.4764e-02,  9.1543e-01],
         [-9.7898e-01,  3.6038e-01,  1.1290e-01,  ..., -1.9539e-03,
          -8.0018e-01, -6.8025e-04],
         [-9.9544e-01,  8.0836e-01,  9.0283e-01,  ..., -2.8582e-03

hiddens tensor([[[-1.2180e-01,  9.0310e-01,  8.6093e-01,  ...,  5.9496e-01,
          -2.2308e-01,  9.1276e-01]],

        [[-2.8835e-01, -1.9147e-02,  4.9706e-02,  ..., -2.0869e-03,
          -5.5503e-01,  7.1799e-01]],

        [[-6.3431e-01,  9.7145e-01,  9.8360e-01,  ...,  7.0915e-01,
          -4.9674e-04,  9.0450e-01]],

        ...,

        [[-8.3988e-01, -2.6377e-02, -7.4990e-03,  ..., -1.1165e-03,
          -6.9113e-01,  2.9245e-01]],

        [[ 1.1021e-01, -2.5557e-03, -5.5696e-04,  ...,  2.0419e-02,
          -9.6110e-01, -1.4873e-03]],

        [[-8.5427e-01,  9.0452e-01,  9.8150e-01,  ...,  1.4794e-01,
           4.5746e-04,  9.0951e-01]]], device='cuda:0',
       grad_fn=<CudnnRnnBackward>)
states (tensor([[[-1.2180e-01,  9.0310e-01,  8.6093e-01,  ...,  5.9496e-01,
          -2.2308e-01,  9.1276e-01],
         [-2.8835e-01, -1.9147e-02,  4.9706e-02,  ..., -2.0869e-03,
          -5.5503e-01,  7.1799e-01],
         [-6.3431e-01,  9.7145e-01,  9.8360e-01,  ...,  7.0915e-01

hiddens tensor([[[-4.0353e-01,  9.6818e-01,  9.3029e-01,  ...,  7.5075e-01,
          -2.7604e-01,  9.5604e-01]],

        [[-8.7997e-01,  7.2327e-02,  1.5408e-01,  ..., -4.5230e-04,
          -7.1947e-01,  6.3468e-01]],

        [[-8.8988e-01,  9.9054e-01,  9.9568e-01,  ...,  7.5219e-01,
          -6.5565e-03,  9.4264e-01]],

        ...,

        [[-9.7107e-01,  2.7309e-02,  2.4038e-02,  ..., -3.2544e-04,
          -8.0105e-01,  1.7288e-01]],

        [[-1.0287e-01, -1.8460e-03, -6.1454e-04,  ...,  9.0938e-03,
          -9.7795e-01, -7.0767e-04]],

        [[-9.8043e-01,  9.6696e-01,  9.9635e-01,  ...,  1.1134e-01,
           1.2192e-04,  9.3513e-01]]], device='cuda:0',
       grad_fn=<CudnnRnnBackward>)
states (tensor([[[-4.0353e-01,  9.6818e-01,  9.3029e-01,  ...,  7.5075e-01,
          -2.7604e-01,  9.5604e-01],
         [-8.7997e-01,  7.2327e-02,  1.5408e-01,  ..., -4.5230e-04,
          -7.1947e-01,  6.3468e-01],
         [-8.8988e-01,  9.9054e-01,  9.9568e-01,  ...,  7.5219e-01

In [20]:
im2p.save_model(22.2373046875,"model_small")

Saved Model in model_small


In [7]:
# Randomly sample a caption length, and sample indices with that length.
(images, image_id, target, prob) = next(iter(data_loader))

In [8]:
images.shape

torch.Size([3, 3, 224, 224])

In [9]:
target.shape

torch.Size([3, 6, 18])

In [10]:
prob

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])

In [45]:
# Specify the dimensionality of the image embedding.
embed_size = 256

#-#-#-# Do NOT modify the code below this line. #-#-#-#

# Initialize the encoder. (Optional: Add additional arguments if necessary.)
encoder = EncoderCNN(embed_size)

# Move the encoder to GPU if CUDA is available.
encoder.to(device)
    
# Move last batch of images (from Step 2) to GPU if CUDA is available.   
images = images.to(device)

# Pass the images through the encoder.
features = encoder(images)

print('type(features):', type(features))
print('features.shape:', features.shape)

# Check that your encoder satisfies some requirements of the project! :D
assert type(features)==torch.Tensor, "Encoder output needs to be a PyTorch Tensor." 
assert (features.shape[0]==batch_size) & (features.shape[1]==embed_size), "The shape of the encoder output is incorrect."

TypeError: super(type, obj): obj must be an instance or subtype of type

In [159]:
from model import SentenceRNN

In [160]:
# Specify the dimensionality of the image embedding.
hiddem_size = 256

#-#-#-# Do NOT modify the code below this line. #-#-#-#

# Initialize the encoder. (Optional: Add additional arguments if necessary.)
sentRnn = SentenceRNN(256,256,256)

# Move the encoder to GPU if CUDA is available.
sentRnn.to(device)
    
# Move last batch of images (from Step 2) to GPU if CUDA is available.   
images = images.to(device)

# Pass the images through the encoder.
features = encoder(images)

print('type(features):', type(features))
print('features.shape:', features.shape)

probs, topic, hiddens = sentRnn(features = features)

type(features): <class 'torch.Tensor'>
features.shape: torch.Size([3, 256])


In [41]:
probs.shape

torch.Size([3, 2])

In [40]:
topic.shape

torch.Size([3, 256])

In [38]:
hiddens.shape

torch.Size([3, 256])

In [15]:
from model import WordRNN

In [19]:
target[0].shape

torch.Size([6, 18])

In [154]:
from model import WordRNN, SentenceRNN

In [59]:
topic.shape

torch.Size([3, 1, 256])

In [86]:
target_0.shape

torch.Size([3, 18])

In [155]:
# Specify the number of features in the hidden state of the RNN decoder.
hidden_size = 512

#-#-#-# Do NOT modify the code below this line. #-#-#-#

# Store the size of the vocabulary.
vocab_size = len(vocab)

# Initialize the decoder.
wordRnn = WordRNN(embed_size, hidden_size, vocab_size)

# Move the decoder to GPU if CUDA is available.
wordRnn.to(device)
 
# Move last batch of captions (from Step 1) to GPU if CUDA is available 
target_0 = target[:,0,:].to(device)
topic = topic.to(device)
print(target_0.shape)
print(topic.shape)

# Pass the encoder output and captions through the decoder.
outputs = wordRnn(topic, target_0)

print('type(outputs):', type(outputs))
print('outputs.shape:', outputs.shape)

# Check that your decoder satisfies some requirements of the project! 
assert type(outputs)==torch.Tensor, "Decoder output needs to be a PyTorch Tensor."
assert (outputs.shape[0]==batch_size) & (outputs.shape[1]==target_0.shape[1]) & (outputs.shape[2]==vocab_size), "The shape of the decoder output is incorrect."
 

torch.Size([3, 18])
torch.Size([3, 1, 256])
topics torch.Size([3, 1, 256])
embedding torch.Size([3, 17, 256])
inner inputs torch.Size([3, 18, 256])
type(outputs): <class 'torch.Tensor'>
outputs.shape: torch.Size([3, 18, 4667])


In [70]:
outputs.shape

torch.Size([3, 18, 4667])

In [54]:
target_0.shape

torch.Size([6, 18])

In [101]:
s_max = 6
n_max = 50
states = None

In [175]:
for i in range(s_max):
    p, topic, states = sentRnn.sample(features, states)
    samples_ids = wordRnn.sample(topic, max_len=n_max)
    print(p)
    p = (p > 0.5).squeeze(1)
    print(p.shape)
    print(p)
    print(samples_ids.shape)
    print(p[:,0].view(3,1))
    print(torch.Tensor(samples_ids).to(device))
    samples_ids = samples_ids * p[:,0].cpu().data.numpy().reshape(3,1)
    print(samples_ids)
    break

torch.Size([3, 1, 2])
tensor([[[0.5179, 0.4821]],

        [[0.4756, 0.5244]],

        [[0.4941, 0.5059]]], device='cuda:0', grad_fn=<SoftmaxBackward>)
torch.Size([3, 2])
tensor([[1, 0],
        [0, 1],
        [0, 1]], device='cuda:0', dtype=torch.uint8)
(3, 50)
tensor([[1],
        [0],
        [0]], device='cuda:0', dtype=torch.uint8)
tensor([[2589., 2437., 1423.,  540.,  618., 4498.,  577.,  140.,  701., 2128.,
         3688.,   96., 3841., 4473., 1564., 2486., 1455., 4657., 4593., 2525.,
         4358., 3038., 3270., 2296., 2296., 3365., 1014., 2012.,  275., 2894.,
         4281., 2814.,  956., 4091., 1694., 3925., 3571., 1751., 4643., 2490.,
         2490., 4296., 1930., 1500., 3848., 1691., 2791., 1118., 4214., 1244.],
        [2589., 2437., 1423.,  540.,  618., 4498.,  577.,  140.,  701., 2128.,
         3688.,   96., 3841., 4473., 1564., 2486., 1455., 4657., 4593., 2525.,
         4358., 3038., 3270., 2296., 2296., 3365., 1014., 2012.,  275., 2894.,
         4281., 2814.,  95